<a href="https://colab.research.google.com/github/jmhuer/utaustin_optimization/blob/main/homework7/ETC_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Explore then Commit (ETC)

In this excercises, we will be playing with the Multi-arm bandit problem with Explore then Commit algorithm.

## Setup

Consider unstructural bandit problem. Suppose we have $k$ arms, each with random rewards $p_i = u_i + \epsilon$ where $\epsilon$ is draw from i.i.d. standard gaussian. (Note that we only require $\epsilon$ to be sub-gaussian for the analysis to go through)

The following codes is capturing the setup.

In [2]:
import numpy as np
import pdb
import matplotlib.pyplot as plt
import plotly.graph_objects as graph

class Gaussian_Arm:
  def __init__(self, num_arms, mu=None):
    '''
    num_arms: (int). the number of arms
    mu: (None or list-type). the mean of the reward of each arm.
        if set to None, a random vector will be generated.
    '''
    if num_arms <= 1 or not isinstance(num_arms, int):
      print('number of arms has an int that is at least two')
      return
    
    self.num_arms = num_arms
    #
    if mu:
      self.mu = np.asarray(mu)
      if len(self.mu) != num_arms:
        print('The lenth of mu does not match the number of arms')
        return
    else:
      self.mu = np.random.rand((num_arms))
    # 
    self.delta = max(self.mu) - min(self.mu)
    #

    # keep track of the rewards for the user
    self.rewards_history = []
    # keep track of how many times the arms have been pulled
    self.total_pull = 0 

  def pull_arm(self, arm_id=-1, pull_time=1):
    if arm_id < 0 or arm_id >= self.num_arms:
      print('please specify arm id in the range of 0-%d' % (self.num_arms))
      return
    assert (isinstance(pull_time, int) and pull_time >= 1)
    self.total_pull += pull_time
    # Generate reward
    reward = self.mu[arm_id] * pull_time + sum(np.random.randn(pull_time))
    self.rewards_history.append(reward)
    return reward


  def genie_reward(self):
    '''
    the best expected reward after pulling self.total_pull times
    '''
    best_mu = max(self.mu)
    return self.total_pull * best_mu

  def my_rewards(self):
    return sum(self.rewards_history)

  def clear_reward_hist(self):
    self.rewards_history = []
    self.total_pull = 0


## Algorithm review

(Please refer to the lecture notes and the text book for details)

The parameter to set: the exploration time m\*k

1. Play each arm in the round-robin fashion until each arm are played m times
2. Compute the empirical reward estimation for each arm
3. Play the best arm (according to the empirical reward estimation) until the end of the game

## Goal of these exercises

Implement the following:

1. Basic ETC algorithm implementation
2. Plot the expected regret of ETC VS horizon ($n$).
3. The doubling trick.
4. Plot the expected regret of ETC with doubling trick VS horizon ($n$).

Answer the following:

1. What are the pros and cons of the doubling trick?
2. Does the regret VS horizon plot looks like $\log n$?
3. What if the exploration time is not set appropriately  (which means, we explore too little or too much) (Usually because the sub-optimality gap cannot be estimated appropriately)?
4. (Open-ended) Can we improve ETC?

## Tips:
1. The regret is expected to be logarithmic against the horizon. To check if the relationship is logarithmic, one can use the semilogx function in matplotlib.pyplot
2. When the regret is not logarithmic, please check against the analysis, and obtain insights there for debugging.
3. To see a smooth curve, one would have to repeat the simulation for multiple time, and take the empirical mean of the regret. This may be slow, if implemented on a single process. So please try-out parallel implementation, if you are comfortable with it (the simulation is very parallelable).


In [140]:
def plot_history(all_history:dict, x:str, y:str , title:str , log = False):
  fig = graph.Figure(layout = graph.Layout(title=graph.layout.Title(text=title)))
  for i in all_history:
      fig.add_trace(graph.Scatter(x    = all_history[i][x],
                                  y    = all_history[i][y],
                                  name = i))
  if log: fig.update_xaxes(type="log")
  fig.show()


def etc_algorithm(arm:Gaussian_Arm, horizon:int, m):
  history = {"step":[], "regret":[], "expected_mu":np.zeros((arm.num_arms)), "arm_chosen":[],"N":np.zeros((arm.num_arms))}
  m = m(horizon, arm.delta)
  for i in range(horizon):
    ##Phase 1 : Explore and record experimental muo
    if i <= arm.num_arms * m:
      arm_decision = i % arm.num_arms # + 1? 
      reward = arm.pull_arm(arm_decision)
      N = history['N'][arm_decision]
      current_arm_muo = history['expected_mu'][arm_decision] 
      history['expected_mu'][arm_decision] =  (current_arm_muo*N + reward )/(N + 1)
    else:
    ##Phase 2 : Commited stage
      arm_decision = np.argmax(history['expected_mu'])
      reward = arm.pull_arm(arm_decision)
    ## store history
    history['N'][arm_decision]+=1
    history['step'].append(i)
    history['regret'].append( float(arm.genie_reward() - arm.my_rewards())  )
    history['arm_chosen'].append(arm_decision)
  print("experimental mu:", history['expected_mu'])
  return history




In [141]:
##initialize arms 
arm = Gaussian_Arm(2, mu =  [1.2, 1.4] )
total_run = 4072
m = lambda total_run, delta: max(1,((4/delta**2)*np.log((total_run*delta**2)/4)))
print("horizon: {} \t m: {} \t no_arms: {}".format(total_run,  m(total_run, arm.delta), arm.num_arms))


# average regret for n runs 
mean_history = {"step": list(range(total_run)), 
                "regret": np.zeros(total_run)}
                
for i in range(5):
  history = etc_algorithm(arm, horizon= total_run, m = m)
  mean_history["regret"] = (mean_history["regret"]*i + history["regret"]) /((i + 1)*len(mean_history["regret"]) )



# combine all plots into one
all_history = {"ETC": mean_history}
#plot
plot_history(all_history, x="step" , y="regret", title="e-greedy v log horizon", log=True)
plot_history(all_history, x="step" , y="regret", title="e-greedy v horizon", log=False)

horizon: 4072 	 m: 370.6719372242269 	 no_arms: 2
experimental mu: [1.22210032 1.38213985]
experimental mu: [1.26358378 1.45562858]
experimental mu: [1.20664634 1.42415142]
experimental mu: [1.13248454 1.33414417]
experimental mu: [1.1955017  1.43192911]


In [146]:
def doubling_etc_algorithm(arm:Gaussian_Arm, rounds:int, m):
  history = {"step":[], "regret":[], "expected_mu":np.zeros((arm.num_arms)), "arm_chosen":[],"N":np.zeros((arm.num_arms))}
  for i in range(1+1,rounds):
    n = 2**i - 2
    mval = m(n, arm.delta)
    for i in range(n):
        ##Phase 1 : Explore and record experimental muo
        if i <= arm.num_arms * mval:
          arm_decision = i % arm.num_arms # + 1? 
          reward = arm.pull_arm(arm_decision)
          N = history['N'][arm_decision]
          current_arm_muo = history['expected_mu'][arm_decision] 
          history['expected_mu'][arm_decision] =  (current_arm_muo*N + reward )/(N + 1)
        else:
        ##Phase 2 : Commited stage
          arm_decision = np.argmax(history['expected_mu'])
          reward = arm.pull_arm(arm_decision)
        ## store history
        history['N'][arm_decision]+=1
        history['step'].append(i)
        history['regret'].append( float(arm.genie_reward() - arm.my_rewards())  )
        history['arm_chosen'].append(arm_decision)
  print("experimental mu:", history['expected_mu'])
  return history



In [147]:
##initialize arms 
arm = Gaussian_Arm(2, mu =  [1.2, 1.4] )
rounds = 12
total_run = sum([2**i - 2 for i in range(1,rounds)])
m = lambda total_run, delta: max(1,((4/delta**2)*np.log((total_run*delta**2)/4)))
print("horizon: {} \t m: {} \t no_arms: {}".format(total_run,  m(100, arm.delta), arm.num_arms))


# average regret for n runs 
mean_history = {"step": list(range(total_run)), 
                "regret": np.zeros(total_run)}
for i in range(10):
  history = doubling_etc_algorithm(arm, rounds= rounds, m = m)
  mean_history["regret"] = (mean_history["regret"]*i + history["regret"]) /( (i + 1)*len(mean_history["regret"]) )


# combine all plots into one
all_history = {"ETC": mean_history}
#plot
plot_history(all_history, x="step" , y="regret", title="e-greedy v log horizon", log=True)
plot_history(all_history, x="step" , y="regret", title="e-greedy v horizon", log=False)

horizon: 4072 	 m: 1 	 no_arms: 2
experimental mu: [1.20524746 1.39764473]
experimental mu: [1.13027528 1.33784747]
experimental mu: [1.24204565 1.38235599]
experimental mu: [1.198657   1.49522827]
experimental mu: [1.22527241 1.39263253]
experimental mu: [1.20027276 1.48023038]
experimental mu: [1.18840384 1.32724879]
experimental mu: [1.19910466 1.46312477]
experimental mu: [1.33084465 1.4845874 ]
experimental mu: [1.32384065 1.48353793]
